In [13]:
t0_df = spark.read.csv("sample-data/train_sample.csv",header=True,inferSchema=True)
t0_df.printSchema()

root
 |-- ip: integer (nullable = true)
 |-- app: integer (nullable = true)
 |-- device: integer (nullable = true)
 |-- os: integer (nullable = true)
 |-- channel: integer (nullable = true)
 |-- click_time: timestamp (nullable = true)
 |-- attributed_time: timestamp (nullable = true)
 |-- is_attributed: integer (nullable = true)



In [14]:
t0_df.count()

100000

In [15]:
t0_df = t0_df.sample(False,0.5)
t0_df.count()

50143

In [16]:
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer

final_df = t0_df

vectorAssembler = VectorAssembler(inputCols=[
    "ip",
    "app",
    "device",
    "os",
    "channel"
    #"click_time", TimeStamp is not supported. Need to get important data from these
    #"attributed_time"
], outputCol="features")

v_final_df = vectorAssembler.transform(final_df)


In [17]:
splits = v_final_df.randomSplit([0.6,0.4],1)

train_df = splits[0]
test_df = splits[1]

print(train_df.count(),test_df.count())

30120 20023


In [20]:
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder, CrossValidator
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

dt = DecisionTreeClassifier(labelCol="is_attributed",featuresCol="features")

dt_model = dt.fit(train_df)

predictions_df = dt_model.transform(test_df)

evaluator = BinaryClassificationEvaluator(
    labelCol="is_attributed",
    rawPredictionCol="prediction",
    metricName="accuracy")

accuracy = evaluator.evaluate(predictions_df)
accuracy

IllegalArgumentException: 'BinaryClassificationEvaluator_4d229e4672397b033660 parameter metricName given invalid value accuracy.'

In [ ]:
#test_df = spark.read.csv("sample-data/test.csv",header=True,inferSchema=True)
